In [1]:
!python3 -m pip install --upgrade pip

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
!pip install nougat-ocr

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
!pip3 install openai==0.27.8

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
!pip3 install python-dotenv tiktoken

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
!pip install PyPDF2

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
!pip install pdf2image

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 3 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Ign:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
Err:1 http://security.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.1
  404  Not Found [IP: 91.189.91.82 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/main/p/poppler/poppler-utils_22.02.0-2ubuntu0.1_amd64.deb  404  Not Found [IP: 91.189.91.82 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [8]:
import re

# 取得したいURL
url = "http://arxiv.org/pdf/2309.11688v1"
identifier = re.search(r'/([^/]+)$', url).group(1)
identifier

'2309.11688v1'

In [9]:
nougat_path = identifier+".pdf"
nougat_path

'2309.11688v1.pdf'

In [10]:
import requests

def download_pdf(link, save_path):
    response = requests.get(link)
    with open(save_path, 'wb') as file:
        file.write(response.content)

In [11]:
import os

files_to_remove = ['sample.pdf', 'sample.mmd']

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"{file} を削除しました。")
    else:
        print(f"{file} は存在しません。")

sample.pdf を削除しました。
sample.mmd を削除しました。


In [12]:
url = "https://arxiv.org/pdf/" + identifier

# download_pdf(url, nougat_path)
download_pdf(url, "sample.pdf")

In [13]:
# nougat_pdf_path = "./" + nougat_path

In [14]:
!nougat ./sample.pdf --out .

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0%|                                                     | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:459: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,
INFO: likely hallucinated title at the end of the page: ## Appendix C Appendix C Tool Input Prompt
INFO:root:Processing file sample.pdf with 9 pages
100%|█████████████████████████████████████████████| 2/2 [00:13<00:00,  6.63s/it]


In [15]:
# import subprocess

# command = ["nougat", "./2309.05557v2.pdf", "--out", "."]
# result = subprocess.run(command, capture_output=True, text=True)

# # 結果を表示
# print(result.stdout)

In [16]:
# nougat_txt_path = identifier+".mmd"
nougat_txt_path = "sample"+".mmd"

In [17]:
import openai

In [18]:
from dotenv import load_dotenv
import os

In [19]:
load_dotenv()

True

In [20]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [21]:
# Step 1: Reading the uploaded text file

with open(nougat_txt_path, "r", encoding="utf-8") as file:
    paper_content = file.read()

In [22]:
def summary_abstract(prompt_text):
    messages = [
        # {"role": "system", "content": prompt},
        {"role": "user", "content": f"{prompt_text}"}
    ]
    res = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        temperature=0.8
    )

    return res["choices"][0]["message"]["content"]

In [23]:
import numpy as np
# APIへ送信するワード数の上限を設定する
# WC_MAX = int(4096 * 7 / 16)    # 1792
WC_MAX = int(2048 * 8 / 16)



# テキスト情報から単語途中で現れるハイフンと改行を削除する
def clean_extracted_text(text) :
    text = text.replace('-\n', '')
    text = re.sub(r'\s+', ' ', text)
    return text


# APIへ送信するメッセージ（プロンプト）を作成する
def create_prompt(text, total_num_req) :
    return f"""英語の研究論文の一部を日本語で要約するタスクを行います。
研究論文は全部で{total_num_req + 1}個に分割しています。
以下のルールに従ってください。

・リスト形式で出力する (先頭は - を使う)
・簡潔に表現する
・不明な単語や人名と思われるものは英語のまま表示する

それでは開始します。

英語の論文の一部:
{text}

日本語で要約した文章:"""

# テキストをWC_MAXごとに分割する
def split_text(text, wc_max) :
    '''
    :param text: english oneline sentence
    :param wc_max:  word count max (if token max is 4097, it should be 2048)
    :return: chunk[] 
    '''
    words = text.split()  # Split the text into words
    chunks = [' '.join(words[i :i + wc_max]) for i in range(0, len(words), wc_max)]
    return chunks

In [24]:
# PDFファイルからテキスト情報を抽出して整形する
extracted_text = paper_content
clean_text = clean_extracted_text(extracted_text)

# テキスト情報の概要を表示する。APIコール回数を表示する。
print(f"input file has {len(clean_text.split())} words, "
      f"so API requests repeat {np.ceil(len(clean_text.split()) / WC_MAX)} times.")

# 分割処理（4,097トークン以下に抑えるための処理）
splited_clean_text = split_text(clean_text, WC_MAX)


input file has 4263 words, so API requests repeat 5.0 times.


In [25]:
# 分割したテキストごとに要約をリクエストする
for i in range(len(splited_clean_text)) :
    print(f"{i} times request... ({len(splited_clean_text[i].split())} words are included)")
    # Setup prompt messages
    messages = [
        {"role" : "system",
         "content" : "あなたは、データ分析を専門に行う研究者です。あなたは、前後に問い合わせした内容を考慮して思慮深い回答をステップバイステップで行います。In general, try to **make plans** with as few steps as possible. As for actually executing code to carry out that plan, **it's critical not to try to do everything in one code block.** You should try something, print information about it, then continue from there in tiny, informed steps. You will never get it on the first try, and attempting it in one go will often lead to errors you cant see.You are capable of **any** task."},
        {"role" : "user", "content" : create_prompt(splited_clean_text[i], len(splited_clean_text))}
    ]

    # APIにリクエストを送信する
    # token数超過などの例外を返す可能性があるため、try/exceptを設定しておく。
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
            temperature=0.8,
            max_tokens=800,  # 生成するトークンの最大数（入力と合算して4097以内に収める必要あり）
            n=1,  # 生成するレスポンスの数
            stop=None,  # 停止トークンの設定\
            top_p=1,  # トークン選択時の確率閾値
        )
        # 生成するレスポンス数は1という前提
        print(response["choices"][0]["message"]["content"])
    except openai.error.InvalidRequestError as e:
        print("OpenAI API error occurred: " + str(e))

0 times request... (1024 words are included)
- 大規模言語モデル（LLM）は、質問応答タスクで素晴らしいパフォーマンスを示していますが、モデルのトレーニングデータに含まれていない知識や現実世界との直接の観察や対話を通じてのみ獲得できる知識が必要な質問では、そのパフォーマンスが制限されます。
- 既存の方法は、連続的に呼び出されるモジュールの使用によって推論タスクを分解していますが、深い推論タスクに対する能力が制限されています。
- Recursion based extensible LLM（REBEL）という手法を導入し、動的な計画や前向き推論戦略などの自動推論技術を使用して、オープンワールドの深い推論タスクを扱います。
- REBELは、LLMが再帰的な問題分解と外部ツールの利用を通じて推論できるようにします。REBELが使用するツールは、自然言語の説明で指定されます。
- REBELの能力を示すために、複合的かつ対話的な環境で外部ツールの深く入れ子になった使用を必要とする問題のセットでREBELの機能を示します。

- LLMを使用した推論タスクにアプローチする方法は、プロンプトエンジニアリング技術とファインチューニング技術、またはそれらの組み合わせに分類できます。
- REBELアルゴリズムを導入し、質問を再帰的に解決する方法、質問をサブ質問に分割する方法、ツールを選択する方法、ツールを使用する方法などを説明します。
- REBELシステムには、利用可能なツールのリストがあり、各質問に必要なツールが決定されます。
1 times request... (1024 words are included)
- アルゴリズム1では、質問にツールを使用せずに回答できるかどうかを確認します。
- ツールピッカーでは、質問に対する回答を決定するために、ツールリストの中から最適なツールを選びます。
- ツールの入力生成では、GPT-3を使用してツールの入力を生成します。ツールの説明と動的パラメータを提供します。
- ツールの使用では、動的パラメータと静的パラメータを使用してAPIリクエストを作成し、回答を生成します。
- REBELは、多様な知識ベースからの質問に一貫して回答できる能力を持つ最先端のシステムです。
- REBE

画像の抽出

In [28]:
!pip install fitz

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [29]:
!pip uninstall -y PyMuPDF
!pip install PyMuPDF

Found existing installation: PyMuPDF 1.23.3
Uninstalling PyMuPDF-1.23.3:
  Successfully uninstalled PyMuPDF-1.23.3
  Obtaining dependency information for PyMuPDF from https://files.pythonhosted.org/packages/1f/bc/3392a788fee48ac6869f4611df059bcac96374ff2b29bff814f90687e7bc/PyMuPDF-1.23.3-cp310-none-manylinux2014_x86_64.whl.metadata
  Using cached PyMuPDF-1.23.3-cp310-none-manylinux2014_x86_64.whl.metadata (3.4 kB)
Using cached PyMuPDF-1.23.3-cp310-none-manylinux2014_x86_64.whl (4.3 MB)
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [30]:
!pip install fitz

DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [31]:
import fitz  # PyMuPDF
from PIL import Image

# pdf_path = nougat_path
pdf_path = "sample.pdf"
doc = fitz.open(pdf_path)
extracted_images = []

for page in doc:
    # ページからイメージを抽出
    for img_index in page.get_images(full=True):
        xref = img_index[0]
        base_image = fitz.Pixmap(doc, xref)
        pil_img = Image.frombytes("RGB", [base_image.width, base_image.height], base_image.samples)
        extracted_images.append(pil_img)

# 最初の抽出された図を表示
if extracted_images:
    extracted_images[0].show()
else:
    "このPDFには抽出可能な図が含まれていません。"


In [32]:
for img in extracted_images:
    img.show()